### Using duckdb with Python API

In [1]:
from IPython.display import display, Image

# Replace 'your_image_url' with the actual URL of the image
image_url = 'https://duckdb.org/images/logo-dl/DuckDB_Logo.png'

# Display the image in the notebook
display(Image(url=image_url))

DuckDB is a database management system, and it is useful in the geospatial domain for efficiently storing and retrieving location-based data.

In simpler terms, DuckDB helps organize and manage information related to locations, like maps or geographic data. It's handy for tasks such as tracking objects, analyzing spatial patterns, or finding nearby points on a map. DuckDB makes working with geospatial data in computer programs easier and more efficient.

DuckDB stands out in the geospatial domain because it's faster, lightweight, and often more efficient compared to other geospatial packages.

* Speed: DuckDB is designed to be fast, allowing for quick processing and retrieval of geospatial data. Its optimized architecture enables speedy execution of queries and analyses, making it a favorable choice for applications that require real-time or near-real-time processing of location-based information.

* Lightweight: DuckDB is a lightweight database, meaning it doesn't require significant computational resources. This makes it suitable for use in various environments, including resource-constrained devices or systems where efficient resource utilization is crucial. The lightweight nature of DuckDB can contribute to faster deployment and lower operational costs.

* Efficiency: DuckDB is built with efficiency in mind, providing a balance between performance and resource utilization. Its design allows for quick data retrieval and processing, making it a reliable option for applications dealing with large geospatial datasets. The efficiency of DuckDB can result in improved overall system performance.

* Integration: DuckDB is designed to integrate seamlessly with programming languages commonly used in data science and geospatial analysis, such as Python. This makes it easier for developers and data scientists to incorporate DuckDB into their workflows, benefiting from its speed and efficiency while leveraging familiar programming tools.

### Objective of this task

* DucDB Exploration:
   - Creating database in duckdb
   - Loading table into the database
   - Loading bulk data inot the database
* Spatial Analysis Techniques:
   - Performining Buffer with duckdb
   - Making spatial Selection with duckdb
* Data Visualization:
   - Using leafmap for Data Visualization
   - Exploring geopandas for data visualization trying the gpd.explore() function.

installing the required package,uisng pip pip inside a notebook is not a standard pratcice it is better you installed all the required libary in a virtual env, start the notebook within the environement to run your analysis. Incase you want to install the libary within the notebook you can use **%pip install** and the name of the libary you want to install

In [2]:
# %pip install duckdb leafmap

import the neccessary libraries

In [3]:
#import the libaries
import duckdb
import pandas as  pd
import leafmap
import os
import pyogrio
import geopandas as gpd

Import jupysql Jupyter extension to create SQL cells

This enables you to write sql query in your notebook, you can use **%sql** when writeing just a line of sql or **%%sql** when writing multiple line of sql in your notebook cell

In [4]:
%load_ext sql

In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

To download the data used in this analysis

In [6]:
# url = r'https://download.geofabrik.de/africa/nigeria-latest-free.shp.zip'
# leafmap.download_file(url,unzip=True)

set folder path

In [7]:
nigeria_osm = 'nigeria-latest-free_shp'
nigeria_admin = 'NGA_adm'
data_folder = 'data'

In [8]:
# load the data in the osm nigeria shapefile folder
data_folder = os.path.join(data_folder, nigeria_osm)
data = os.listdir(data_folder)
for item in data:
    if item.endswith('.shp'):
        print(item)

gis_osm_buildings_a_free_1.shp
gis_osm_landuse_a_free_1.shp
gis_osm_natural_a_free_1.shp
gis_osm_natural_free_1.shp
gis_osm_places_a_free_1.shp
gis_osm_places_free_1.shp
gis_osm_pofw_a_free_1.shp
gis_osm_pofw_free_1.shp
gis_osm_pois_a_free_1.shp
gis_osm_pois_free_1.shp
gis_osm_railways_free_1.shp
gis_osm_roads_free_1.shp
gis_osm_traffic_a_free_1.shp
gis_osm_traffic_free_1.shp
gis_osm_transport_a_free_1.shp
gis_osm_transport_free_1.shp
gis_osm_waterways_free_1.shp
gis_osm_water_a_free_1.shp


Check the nigeria admin shapeifle that was added to data

In [14]:
# load the data in the osm nigeria shapefile folder
data_folder = os.path.join('data',nigeria_admin)
data = os.listdir(data_folder)
for item in data:
    if item.endswith('.shp'):
        print(item)

NGA_adm0.shp
NGA_adm1.shp
NGA_adm2.shp


Connecting to Duckdb

Create a db for nigeria where all the data will be store before analysis

In [15]:
con = duckdb.connect("nigeria.db")

Install and load spatial extension

In [16]:
con.install_extension('spatial')
con.load_extension('spatial')

When you first run con.sql("SHOW TABLE") it should show you an empty database without any table present becuase we just created the database untill you create or write a table to the database, it will be empty.

In [17]:
con.sql("SHOW TABLES")

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ nigeria_state │
└───────────────┘

Load the Nigeria Admin shapefile to the database

By default DuckDB operates on an in-memory database. That means that any tables that are created are not persisted to disk. Using the .connect method a connection can be made to a persistent database. Any data written to that connection will be persisted, and can be reloaded by re-connecting to the same file.

In [19]:
%%timeit
# lets try it with one and see then later we can use a loop to load the rest of the data into the database
nga_admin1  = 'NGA_adm1.shp'
nga_state_data = os.path.join('data',nigeria_admin, nga_admin1)
# read the data into a dataframe
# nga_state_gdf = gpd.read_file(nga_state_data)
# print(nga_state_gdf)

# pass the data into the database
# create a new table from the contents of a DataFrame
query = f"DROP TABLE IF EXISTS nigeria_state; \
        CREATE TABLE nigeria_state AS SELECT * FROM  ST_Read('{nga_state_data}')"
con.execute(query)

# if the table already exit all we need to do is to insert data into the table
# insert into an existing table from the contents of a DataFrame
#con.execute("INSERT INTO existing_table SELECT * FROM loaded_Dataframe")

356 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
con.sql("SHOW TABLES")

┌───────────────┐
│     name      │
│    varchar    │
├───────────────┤
│ nigeria_state │
└───────────────┘

Lets query the data using the State name

In [21]:
# query the table
con.table('nigeria_state').show()

┌───────┬─────────┬─────────┬───┬───────────┬───────────┬──────────────────────┐
│ ID_0  │   ISO   │ NAME_0  │ … │ NL_NAME_1 │ VARNAME_1 │         geom         │
│ int64 │ varchar │ varchar │   │  varchar  │  varchar  │       geometry       │
├───────┼─────────┼─────────┼───┼───────────┼───────────┼──────────────────────┤
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((7.508012…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((13.72386…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ MULTIPOLYGON (((7.…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((6.915181…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((10.73444…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ MULTIPOLYGON (((6.…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((8.869718…  │
│   163 │ NGA     │ Nigeria │ … │ NULL      │ NULL      │ POLYGON ((13.36041…  │
│   163 │ NGA     │ Nigeria 

In [22]:
con.sql('SELECT * EXCLUDE geom ,ST_AStext(geom) as geomery FROM nigeria_state').df().head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geomery
0,163,NGA,Nigeria,1,Abia,State,State,None,None,"POLYGON ((7.508012771606559 6.009689807891846,..."
1,163,NGA,Nigeria,2,Adamawa,State,State,None,None,POLYGON ((13.723860740661621 10.91172790527349...
2,163,NGA,Nigeria,3,Akwa Ibom,State,State,None,None,MULTIPOLYGON (((7.610694885253963 4.4729170799...
3,163,NGA,Nigeria,4,Anambra,State,State,None,None,"POLYGON ((6.915181159973201 6.711037158966064,..."
4,163,NGA,Nigeria,5,Bauchi,State,State,None,None,POLYGON ((10.734446525573787 12.40430355072015...


In [24]:
%%time
osm_building  = 'gis_osm_buildings_a_free_1.shp'
building_data = os.path.join('data', nigeria_osm, osm_building)
# get the first 10 building data
con.sql(f"SELECT * FROM ST_Read('{building_data}') LIMIT 10")

Wall time: 440 ms


┌──────────┬───────┬──────────┬───┬─────────┬──────────────────────┐
│  osm_id  │ code  │  fclass  │ … │  type   │         geom         │
│ varchar  │ int32 │ varchar  │   │ varchar │       geometry       │
├──────────┼───────┼──────────┼───┼─────────┼──────────────────────┤
│ 10561268 │  1500 │ building │ … │ NULL    │ POLYGON ((3.397479…  │
│ 28923442 │  1500 │ building │ … │ NULL    │ POLYGON ((7.654457…  │
│ 30047438 │  1500 │ building │ … │ NULL    │ POLYGON ((3.972691…  │
│ 31895041 │  1500 │ building │ … │ NULL    │ POLYGON ((7.720429…  │
│ 223198   │  1500 │ building │ … │ stadium │ POLYGON ((7.427332…  │
│ 38942560 │  1500 │ building │ … │ NULL    │ POLYGON ((7.443189…  │
│ 39041234 │  1500 │ building │ … │ NULL    │ POLYGON ((7.451393…  │
│ 39239269 │  1500 │ building │ … │ NULL    │ POLYGON ((7.402444…  │
│ 39239270 │  1500 │ building │ … │ NULL    │ POLYGON ((7.403506…  │
│ 39239271 │  1500 │ building │ … │ NULL    │ POLYGON ((7.403802…  │
├──────────┴───────┴──────────┴───

Count the total number of building mapped in nigeria

In [26]:
%%time
osm_building  = 'gis_osm_buildings_a_free_1.shp'
building_data= os.path.join('data', nigeria_osm, osm_building)
# get the total numbers of building mapped on osm in Nigeria
query = f"SELECT COUNT(*) FROM ST_Read('{building_data}')"
print(con.sql(query))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     10399995 │
└──────────────┘

Wall time: 7min 25s


In [27]:
%%timeit
# lets try it with one and see then later we can use a loop to load the rest of the data into the database
# osm_building  = 'gis_osm_buildings_a_free_1.shp'
# building_data= os.path.join('data', nigeria_osm, osm_building)
# # read the data into a dataframe

# # pass the data into the database
# # create a new table from the contents of a DataFrame
# query = f"DROP TABLE IF EXISTS osm_nigeria_building; \
#         CREATE TABLE osm_nigeria_building AS SELECT * FROM  ST_Read('{building_data}')"
# con.execute(query)

# if the tbale already exit all will need to do is to insert into the table
# insert into an existing table from the contents of a DataFrame
#con.execute("INSERT INTO existing_table SELECT * FROM loaded_Dataframe")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

7min 17s ± 19.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


show table to check if the osm_nigeria_building has been added to the database

In [28]:
con.sql("SHOW TABLES")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ nigeria_state        │
│ osm_nigeria_building │
└──────────────────────┘

## Lagos Nigeria Analysis

So i tried to load Nigeria data into the db and it was taking too much time, mind you we only need lagos nigeria data for this analsyis, so we will should just use the duckdb spatial extention to clip pout lagos data from all our osm data and store it in lagos db

In [33]:
lagos_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) AS geometry\
                   FROM nigeria_state WHERE NAME_1= 'Lagos' ").df()
lagos_gdf = leafmap.df_to_gdf(lagos_df)

In [35]:
lagos_gdf.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

In [ ]:
# to use the explore function to visualize data,
# folium, mapclassify and 
# %pip install mapclassify

In [ ]:
def clip_with_boundary(data, boundary,table_name, db_name):
    with duckdb.connect(db_name) as con:
        query = f"DROP TABLE {table_name} IF EXISTS; \
        CREATE TABLE {table_name} AS\
             SELECT * \
            FROM {data} \
        JOIN {boundary} \
        ON ST_intersects({data.geom}, {boundary.geom}) \
        "
        con.sql(query)

Load the place of 

In [ ]:
def load_data_db(data,table_name,db_name):
    # using the context manager closes the connection automatically
    with duckdb.connect(db_name) as con:
        query = f"DROP TABLE IF EXISTS {table_name}; \
            CREATE TABLE {table_name} AS SELECT * FROM  ST_Read('{data}')"
        con.sql(query)
    return.table('table_name').show()

In [ ]:
# load the data in the osm nigeria shapefile folder
list_data = []
data_folder = os.path.join(data_folder, nigeria_osm)
data = os.listdir(data_folder)
for item in data:
    if item.endswith('.shp'):
        print(item)
        list_data.append(item)

In [ ]:
# new name that will be assign to each data should be written here
list_tabble_name = []
for %load_ext_ext_ext_ext_ext_extin list_data:
    for table_name in list_tabble_name:
        load_data_db(data,table_name)

use leafmap to visualize the nigeria buidling shapefile

Load data into the databse using sqalchemy method
* read the data using pyogrio, the reason why pyogrio is used to load this data instead of the commonly known geopandas is because of the speed, it is 18x faster than geopandas.
* Use a for loop to load all the data in the nigeria shapefile into the db that was connected to i.e "nigeria.db"

In [ ]:
gpd.to_sql()

In [ ]:
%%timeit
# lets try it with one and see then later we can use a loop to load the rest of the data into the database
osm_building  = 'gis_osm_buildings_a_free_1.shp'
budiling_data = os.path.join(home_folder, nigeria_folder, osm_building)
# read the data into a dataframe
buidling_gdf = pyogrio.read_dataframe(budiling_data)

# pass the data into the database
# create a new table from the contents of a DataFrame
query = f"DROP TABLE IF EXISTS;\
            CREATE TABLE osm_nigeria_builing AS SELECT * FROM  {buidling_gdf}').fetchall()"
print(con.execute(query))

# if the tbale already exit all will need to do is to insert into the table
# insert into an existing table from the contents of a DataFrame
#con.execute("INSERT INTO existing_table SELECT * FROM loaded_Dataframe")

In [ ]:
%%timeit
con.sql("SHOW TABLES")

In [ ]:
%%timeit
# lets try it with one and see then later we can use a loop to load the rest of the data into the database
osm_building  = 'gis_osm_buildings_a_free_1.shp'
building_data = os.path.join(home_folder, nigeria_folder, osm_building)
# pass the data into the database
# create a new table from the contents of a DataFrame
query = f"CREATE TABLE osm_nigeria_builing AS SELECT * FROM ST_Read('{building_data}')"
print(con.execute(query))

# if the tbale already exit all will need to do is to insert into the table
# insert into an existing table from the contents of a DataFrame
#con.execute("INSERT INTO existing_table SELECT * FROM loaded_Dataframe")

Show our table again to know if the data is already ingested into the database

In [ ]:
con.sql("SHOW TABLES")

if that works for one, then we will pass the bulk table (data) into the db

In [ ]:
# load the data in the osm nigeria shapefile folder
home_folder = 'data'
nigeria_folder = 'nigeria-latest-free_shp'
data_folder = os.path.join(home_folder, nigeria_folder)
data = os.listdir(data_folder)
for item in data:
    if item.endswith('.shp'):
        print(item)
        
# pass the data into the database
# create a new table from the contents of a DataFrame
query = f"CREATE TABLE osm_nigeria_builing AS SELECT * FROM ST_Read('{bulding_gdf}')"
print(con.execute(query))

### Spatial Analysis

- select all building that are within Lagos state
- count the total buildings that was mapped in lagos state nigeria out of the building mapped in nigeria
- use leafmap to displace this map and deisgn there legend based on there landuse type "f_class"
- clip osm water body to the lagis boundary
- run a buffer of 100m around the water
- select all the buidling within 100m bufferered
- export it out to another data  format
- visualise the data on a map

In [ ]:
#

In [ ]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [ ]:
%%sql

SELECT * FROM duckdb_extensions();

## References 

* https://duckdb.org/docs/api/python/data_ingestion